# Importing Libraries

In [6]:
import pandas as pd

from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

# Importing Data

In [3]:
p1_gen = pd.read_csv('../../data/p1_gen_dtindex.csv', index_col=0, parse_dates=True)
p1_gen.head()

,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
DATE_TIME,,,,,,
2020-05-15,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0
2020-05-15,4135001,1IF53ai7Xc0U56Y,0.0,0.0,0.0,6183645.0
2020-05-15,4135001,3PZuoBAID5Wc2HD,0.0,0.0,0.0,6987759.0
2020-05-15,4135001,7JYdWkrLSPkdwr4,0.0,0.0,0.0,7602960.0
2020-05-15,4135001,McdE0feGgRqW7Ca,0.0,0.0,0.0,7158964.0


# Prepping Data

In [9]:
pred_gen_og = p1_gen.copy()

In [11]:
pred_gen_jun = pred_gen_og[(pred_gen_og.index.month == 6)]
pred_gen_jun = pred_gen_jun.groupby('DATE_TIME').sum()
pred_gen_jun

,PLANT_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
DATE_TIME,,,,,
2020-06-01 00:00:00,90970022,0.0,0.0,5407.250000,153519480.0
2020-06-01 00:15:00,90970022,0.0,0.0,0.000000,153519480.0
2020-06-01 00:30:00,90970022,0.0,0.0,0.000000,153519480.0
2020-06-01 00:45:00,90970022,0.0,0.0,0.000000,153519480.0
2020-06-01 01:00:00,90970022,0.0,0.0,0.000000,153519480.0
...,...,...,...,...,...
2020-06-17 22:45:00,90970022,0.0,0.0,129571.000000,156142755.0
2020-06-17 23:00:00,90970022,0.0,0.0,129571.000000,156142755.0
2020-06-17 23:15:00,90970022,0.0,0.0,129571.000000,156142755.0


In [14]:
pred_6_13 = pred_gen_jun[pred_gen_jun.index >= '2020-06-13 00:00:00'][['DAILY_YIELD']]
pred_6_13

,DAILY_YIELD
DATE_TIME,
2020-06-13 00:00:00,5683.214286
2020-06-13 00:15:00,0.000000
2020-06-13 00:30:00,0.000000
2020-06-13 00:45:00,0.000000
2020-06-13 01:00:00,0.000000
...,...
2020-06-17 22:45:00,129571.000000
2020-06-17 23:00:00,129571.000000
2020-06-17 23:15:00,129571.000000


# Train-Test Split

In [15]:
eotri = round(pred_6_13.shape[0] * .8)
train_pred_6_13 = pred_6_13[:eotri]
test_pred_6_13 = pred_6_13[eotri:]
print(train_pred_6_13.shape, test_pred_6_13.shape)

(382, 1) (96, 1)


In [21]:
pred_6_13

,DAILY_YIELD
DATE_TIME,
2020-06-13 00:00:00,5683.214286
2020-06-13 00:15:00,0.000000
2020-06-13 00:30:00,0.000000
2020-06-13 00:45:00,0.000000
2020-06-13 01:00:00,0.000000
...,...
2020-06-17 22:45:00,129571.000000
2020-06-17 23:00:00,129571.000000
2020-06-17 23:15:00,129571.000000


In [19]:
train_pred_6_13

,DAILY_YIELD
DATE_TIME,
2020-06-13 00:00:00,5683.214286
2020-06-13 00:15:00,0.000000
2020-06-13 00:30:00,0.000000
2020-06-13 00:45:00,0.000000
2020-06-13 01:00:00,0.000000
...,...
2020-06-16 22:15:00,136429.000000
2020-06-16 22:30:00,136429.000000
2020-06-16 22:45:00,136429.000000


In [20]:
test_pred_6_13

,DAILY_YIELD
DATE_TIME,
2020-06-16 23:30:00,136429.000000
2020-06-16 23:45:00,134079.375000
2020-06-17 00:00:00,6955.232143
2020-06-17 00:15:00,0.000000
2020-06-17 00:30:00,0.000000
...,...
2020-06-17 22:45:00,129571.000000
2020-06-17 23:00:00,129571.000000
2020-06-17 23:15:00,129571.000000


# LSTM NN

In [16]:
n_steps = train_pred_6_13.shape[0]
n_features = train_pred_6_13.shape[1]

In [17]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit(train_pred_6_13[0],)